In [133]:
pip install yfinance

In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import statsmodels.api as sm

from pandas_datareader import data
from matplotlib.ticker import FuncFormatter 
from scipy.stats import norm
from numpy import array
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
# %matplotlib widget

In [171]:
tickers_list = ['BTC-USD', 'ETH-USD','LTC-USD']
# tickers_list = "" Value indicated by top 10 currencies in our portfolio

In [172]:
START_DATE = '2020-01-01'
END_DATE = '2021-12-31'


In [173]:
import yfinance as yf
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
        
    return numpy.array(dataX), numpy.array(dataY)

prices_df = yf.download(tickers_list, start=START_DATE, end=END_DATE, adjusted=True)
prices_df = prices_df['Close']
results = []

[*********************100%***********************]  3 of 3 completed


In [174]:
prices_df['BTC-USD'].iloc[-1]

62686.32421875

In [175]:
original_price = []
for ticker in tickers_list:
  current_price = prices_df[ticker].iloc[-1]
  original_price.append(current_price)
original_price
dataFrames_arr = []

In [176]:
for ticker in tickers_list:

  close_df = prices_df[ticker]


  scaler=MinMaxScaler(feature_range=(0,1))
  close_df=scaler.fit_transform(np.array(close_df).reshape(-1,1))

  training_size=int(len(close_df)*0.65)
  test_size=len(close_df)-training_size
  train_data,test_data=close_df[0:training_size,:],close_df[training_size:len(close_df),:1]


  time_step = 100
  X_train, y_train = create_dataset(train_data, time_step)
  X_test, ytest = create_dataset(test_data, time_step)

  X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

  model= Sequential()
  model.add(LSTM(50,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer='adam') 
  model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=5,batch_size = 32,verbose =1)

  shape_input = test_data.shape[0]-100


  x_input=test_data[shape_input:].reshape(1,-1)

  temp_input=list(x_input)
  temp_input=temp_input[0].tolist()



  lst_output=[]
  n_steps=100
  i=0
  while(i<30):
      
      if(len(temp_input)>100):
          #print(temp_input)
          x_input=np.array(temp_input[1:])
          # print("{} day input {}".format(i,x_input))
          x_input=x_input.reshape(1,-1)
          x_input = x_input.reshape((1, n_steps, 1))
          #print(x_input)
          yhat = model.predict(x_input, verbose=0)
          # print("{} day output {}".format(i,yhat))
          temp_input.extend(yhat[0].tolist())
          temp_input=temp_input[1:]
          #print(temp_input)
          lst_output.extend(yhat.tolist())
          i=i+1
      else:
          x_input = x_input.reshape((1, n_steps,1))
          yhat = model.predict(x_input, verbose=0)
          # print(yhat[0])
          temp_input.extend(yhat[0].tolist())
          # print(len(temp_input))
          lst_output.extend(yhat.tolist())
          i=i+1


  df3=close_df.tolist()
  df3.extend(lst_output)

  df3=scaler.inverse_transform(df3).tolist()
  dataFrames_arr.append(df3)

  results.append(df3[-1])

Epoch 1/5
11/11 [==============================] - 8s 287ms/step - loss: 0.0316 - val_loss: 0.0174
Epoch 2/5
11/11 [==============================] - 2s 172ms/step - loss: 0.0050 - val_loss: 0.0595
Epoch 3/5
11/11 [==============================] - 2s 167ms/step - loss: 0.0052 - val_loss: 0.0098
Epoch 4/5
11/11 [==============================] - 2s 168ms/step - loss: 0.0038 - val_loss: 0.0140
Epoch 5/5
11/11 [==============================] - 2s 168ms/step - loss: 0.0038 - val_loss: 0.0177
Epoch 1/5
11/11 [==============================] - 9s 337ms/step - loss: 0.0114 - val_loss: 0.0211
Epoch 2/5
11/11 [==============================] - 2s 164ms/step - loss: 0.0017 - val_loss: 0.0412
Epoch 3/5
11/11 [==============================] - 2s 167ms/step - loss: 0.0011 - val_loss: 0.0107
Epoch 4/5
11/11 [==============================] - 2s 167ms/step - loss: 8.5677e-04 - val_loss: 0.0081
Epoch 5/5
11/11 [==============================] - 2s 167ms/step - loss: 7.9832e-04 - val_loss: 0.0090
Ep

In [177]:
results

[[77762.31961663032], [3251.6492812168544], [228.03571928314275]]

In [178]:
future_prices = []

for pr in results:
  future_prices.append(pr[0])



In [179]:
array = []
for i in range(len(original_price)):
  eachTicker = []
  eachTicker.append(original_price[i])
  eachTicker.append(future_prices[i])
  array.append(eachTicker)

In [180]:
df = pd.DataFrame(data = array, 
                  index = tickers_list, 
                  columns = ['original_prices','future_prices'])

In [181]:
df['pct_change'] = ((df['future_prices'] - df['original_prices'])/df['original_prices']) * 100

In [226]:
table_with_future_prices = dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True) #i dont know how to run this coz google lab dont have this module

AttributeError: ignored

In [210]:
from datetime import datetime

from datetime import timedelta, date

time_extension = len(df3) - 1

EndDate = prices_df.index[0] + timedelta(days=time_extension)

dateIndexes = pd.date_range(start='2020-01-01', end=EndDate)

In [223]:
import dash_core_components as dcc
import plotly.graph_objs as go
from dash import dcc
graphs_arr = []
for i in range(len(dataFrames_arr)):
  title = 'Future prices of ' + tickers_list[i]
  fig = go.Figure(data=[go.Scatter(x=dateIndexes, y=eachDf)])

  graphs_arr.append(fig)

In [224]:
graphs_arr

[Figure({
     'data': [{'type': 'scatter',
               'x': array([datetime.datetime(2020, 1, 1, 0, 0),
                           datetime.datetime(2020, 1, 2, 0, 0),
                           datetime.datetime(2020, 1, 3, 0, 0), ...,
                           datetime.datetime(2021, 11, 19, 0, 0),
                           datetime.datetime(2021, 11, 20, 0, 0),
                           datetime.datetime(2021, 11, 21, 0, 0)], dtype=object),
               'y': [[4.511590003967285], [4.648940086364746], [4.626200199127197],
                     ..., [178.39640435430167], [177.2515524852551],
                     [176.1293309712802]]}],
     'layout': {'template': '...'}
 }), Figure({
     'data': [{'type': 'scatter',
               'x': array([datetime.datetime(2020, 1, 1, 0, 0),
                           datetime.datetime(2020, 1, 2, 0, 0),
                           datetime.datetime(2020, 1, 3, 0, 0), ...,
                           datetime.datetime(2021, 11, 19, 0, 0),
 

ModuleNotFoundError: ignored